# Setup

In [2]:
pip install -r ../requirements.txt

/bin/bash: /anaconda/envs/jupyter_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple/, https://****@pkgs.dev.azure.com/parsionate-ai/ps-libraries/_packaging/pypi-feed/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Add directory above current directory to path
import sys

sys.path.insert(0, '..')

In [4]:
import pandas as pd
import numpy as np
import os
import gzip
import glob
import json
from tqdm import tqdm

In [5]:
path = "../datasets/flickr30k_images"

# Downloading data

To download the Flickr30k data from Kaggle, ensure you have API credentials set up.

For further information see https://github.com/Kaggle/kaggle-api

In [6]:
!kaggle datasets download -d hsankesara/flickr-image-dataset -p ../datasets --unzip

/bin/bash: /anaconda/envs/jupyter_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)
100%|█████████████████████████████████████▉| 8.16G/8.16G [03:48<00:00, 41.2MB/s]
100%|██████████████████████████████████████| 8.16G/8.16G [03:48<00:00, 38.3MB/s]


In [7]:
# removing duplicate images folder

!rm -rf {os.path.join(path, "flickr30k_images/flickr30k_images")}

/bin/bash: /anaconda/envs/jupyter_env/lib/libtinfo.so.6: no version information available (required by /bin/bash)


# Load data

In [32]:
data = pd.read_csv(os.path.join(path, "results.csv"), sep="|", skipinitialspace=True)
data

,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their ...
1,1000092795.jpg,1,"Two young , White males are outside near many ..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
...,...,...,...
158910,998845445.jpg,0,A man in shorts and a Hawaiian shirt leans ove...
158911,998845445.jpg,1,"A young man hanging over the side of a boat , ..."
158912,998845445.jpg,2,A man is leaning off of the side of a blue and...
158913,998845445.jpg,3,"A man riding a small boat in a harbor , with f..."


# Exploration

## Columns, size...

In [33]:
data.columns

Index(['image_name', 'comment_number', 'comment'], dtype='object')

In [34]:
data.shape

(158915, 3)

In [35]:
data.value_counts(["image_name"])

image_name    
1000092795.jpg    5
459538095.jpg     5
459804826.jpg     5
459778335.jpg     5
4597303045.jpg    5
                 ..
3029715635.jpg    5
3029472296.jpg    5
3029463004.jpg    5
3029411230.jpg    5
998845445.jpg     5
Length: 31783, dtype: int64

# Ground truth

In [36]:
data["path"] = data["image_name"]
data["description"] = data["comment"]

In [37]:
ground_truth = data[["path", "description"]]

## Copy columns for experiments on extended MNN-EM

In [38]:
ground_truth["description2"] = ground_truth["description"]

/tmp/ipykernel_18624/3315395984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth["description2"] = ground_truth["description"]


## Target labels for MNN-EM

In [39]:
ground_truth["label"] = 1

/tmp/ipykernel_18624/3465935695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ground_truth["label"] = 1


In [40]:
ground_truth.head(10)

,path,description,description2,label
0,1000092795.jpg,Two young guys with shaggy hair look at their ...,Two young guys with shaggy hair look at their ...,1
1,1000092795.jpg,"Two young , White males are outside near many ...","Two young , White males are outside near many ...",1
2,1000092795.jpg,Two men in green shirts are standing in a yard .,Two men in green shirts are standing in a yard .,1
3,1000092795.jpg,A man in a blue shirt standing in a garden .,A man in a blue shirt standing in a garden .,1
4,1000092795.jpg,Two friends enjoy time spent together .,Two friends enjoy time spent together .,1
5,10002456.jpg,Several men in hard hats are operating a giant...,Several men in hard hats are operating a giant...,1
6,10002456.jpg,Workers look down from up above on a piece of ...,Workers look down from up above on a piece of ...,1
7,10002456.jpg,Two men working on a machine wearing hard hats .,Two men working on a machine wearing hard hats .,1
8,10002456.jpg,Four men on top of a tall structure .,Four men on top of a tall structure .,1
9,10002456.jpg,Three men on a large rig .,Three men on a large rig .,1


# False samples/Complement

In [41]:
false_samples = ground_truth.apply(np.random.permutation, axis=0)

## Target labels for MNN-EM

In [42]:
false_samples["label"] = 0

In [43]:
false_samples.head(10)

,path,description,description2,label
0,4341820422.jpg,An outdoor ice skating rink full of people .,A man in a long blue cape walks into an old st...,0
1,2250555512.jpg,Girl in red shirt standing over rows of oranges .,A group of people at a parade .,0
2,4603095253.jpg,A city street with market vendors and children...,"A group of dark-skinned , south Asian men sit ...",0
3,3497590041.jpg,A man is asleep sitting up on a plane wearing ...,There are three dogs racing around a muddy tra...,0
4,73987570.jpg,Security officer working outside .,A man and woman smile after being wed .,0
5,114715590.jpg,An Indian man is kneeling on a rug praying .,A man is riding a dirt bike on a steep tilt .,0
6,3268467451.jpg,A man working on a train .,Two young women sit by a fountain .,0
7,310715139.jpg,A sexy blond is sitting on a bicycle wearing l...,A young guy standing on the side of the road t...,0
8,2444073390.jpg,Two dogs are running on the grass .,Two men and a woman are walking down an old co...,0
9,6386880789.jpg,A man playing an instrument underneath some st...,Man in a red baseball cap is grilling fish .,0


# Combine ground truth and complement

In [44]:
dataset = pd.concat([ground_truth, false_samples])

In [45]:
# random shuffle
dataset_final = dataset.sample(frac=1, axis=0).reset_index(drop=True)

In [46]:
dataset_final = dataset_final[["path", "description", "description2", "label"]]

## Set descriptions to lowercase

As per [Zhang et al. (2015)](https://dl.acm.org/doi/10.5555/2969239.2969312)

In [47]:
dataset_final["description"] = dataset_final["description"].str.lower()

In [48]:
dataset_final["description2"] = dataset_final["description2"].str.lower()

In [49]:
dataset_final

,path,description,description2,label
0,431018958.jpg,two teenagers walking down the street with an ...,two asian woman at an archery competition gett...,0
1,1698837338.jpg,performers in costumes are performing at a par...,a brunette wearing eyeglasses and white sweate...,0
2,2427731202.jpg,a woman off her bike looking at the area .,horseback riders file down a wooded path .,0
3,3104400277.jpg,two people on motorbikes .,two people on motorbikes .,1
4,2868575889.jpg,multiple people are hanging on to a pole while...,a man holding a little girl up in the air outs...,0
...,...,...,...,...
317825,5105885364.jpg,a chubby woman with her boyfriend .,two men wearing plastic gloves are making juic...,0
317826,2962977152.jpg,two soccer players rush to kick the ball .,two soccer players rush to kick the ball .,1
317827,3203653158.jpg,an african walks through a field .,an african walks through a field .,1
317828,4897359709.jpg,a woman wearing shades and a coat looks upward .,a man in a white collared shirt plays a red gu...,0


## Remove non-ASCII characters

In [50]:
dataset_final["description"] = dataset_final['description'].str.replace(
    r'[^\x00-\x7F]+', '', regex=True)
dataset_final["description2"] = dataset_final['description2'].str.replace(
    r'[^\x00-\x7F]+', '', regex=True)

In [51]:
dataset_final = dataset_final.dropna()

In [52]:
dataset_final = dataset_final.reset_index(drop=True)

In [53]:
dataset_final

,path,description,description2,label
0,431018958.jpg,two teenagers walking down the street with an ...,two asian woman at an archery competition gett...,0
1,1698837338.jpg,performers in costumes are performing at a par...,a brunette wearing eyeglasses and white sweate...,0
2,2427731202.jpg,a woman off her bike looking at the area .,horseback riders file down a wooded path .,0
3,3104400277.jpg,two people on motorbikes .,two people on motorbikes .,1
4,2868575889.jpg,multiple people are hanging on to a pole while...,a man holding a little girl up in the air outs...,0
...,...,...,...,...
317822,5105885364.jpg,a chubby woman with her boyfriend .,two men wearing plastic gloves are making juic...,0
317823,2962977152.jpg,two soccer players rush to kick the ball .,two soccer players rush to kick the ball .,1
317824,3203653158.jpg,an african walks through a field .,an african walks through a field .,1
317825,4897359709.jpg,a woman wearing shades and a coat looks upward .,a man in a white collared shirt plays a red gu...,0


## Export dataset

In [54]:
dataset_final.to_csv(os.path.join(path, "data.csv"))

In [55]:
dataset_final = pd.read_csv(os.path.join(path, "data.csv"))
dataset_final = dataset_final.drop(['Unnamed: 0'], axis=1)
dataset_final

,path,description,description2,label
0,431018958.jpg,two teenagers walking down the street with an ...,two asian woman at an archery competition gett...,0
1,1698837338.jpg,performers in costumes are performing at a par...,a brunette wearing eyeglasses and white sweate...,0
2,2427731202.jpg,a woman off her bike looking at the area .,horseback riders file down a wooded path .,0
3,3104400277.jpg,two people on motorbikes .,two people on motorbikes .,1
4,2868575889.jpg,multiple people are hanging on to a pole while...,a man holding a little girl up in the air outs...,0
...,...,...,...,...
317822,5105885364.jpg,a chubby woman with her boyfriend .,two men wearing plastic gloves are making juic...,0
317823,2962977152.jpg,two soccer players rush to kick the ball .,two soccer players rush to kick the ball .,1
317824,3203653158.jpg,an african walks through a field .,an african walks through a field .,1
317825,4897359709.jpg,a woman wearing shades and a coat looks upward .,a man in a white collared shirt plays a red gu...,0
